# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk
import time

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [2]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t","\\r","\\r\\n"  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [3]:
def setup_es_client():
    es_client = Elasticsearch()

    index_name = "dpe_extract_text"
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    #"analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [336]:
from generate_dpe_annexes_scripts.utils import strip_accents

sleepers = {'after_bulk':1,
           'after_search':0.2}
def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        if isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    return search_instruction
def search_from_search_dict(es_client,search_dict,index_name):
    s_list = list()
    for label,char_list in  search_dict.items():
        time.sleep(sleepers['after_search'])
        search_instruction =generate_instruction_from_list(char_list)
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)
        a_dict=es_client.search(index=index_name, body=search_body,size=5000000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits],dtype='str')
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)
    return s_all

def search_and_affect(data_to_search,id_col,val_col,search_dict,index_name='dpe_extract_text'):
    es_client = setup_es_client()
    bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
    time.sleep(sleepers['after_bulk'])

    res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

    res_table=res_serie.to_frame('label')

    res_table.index.name = 'id'
    res_table=res_table.reset_index()


# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return res_table
def categorize_search_res(table,label_cat=None,category_dict=None):
    
    if category_dict is not None:
        table['category']=table.label.replace(category_dict)
    
    if label_cat is not None:
        table['label']=pd.Categorical(table.label,categories=label_cat,ordered=True)

    #df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    table.label = table.label.fillna('indetermine')
    return table

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [5]:
from generate_dpe_annexes_scripts.utils import reload_package

In [6]:
import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)

In [333]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables

trtv = DPETrTvTables()

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='94':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
            td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
            td011_p = pd.read_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv', dtype=str)

            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]

            print(dept_dir)
            
            # ENVELOPPE PROCESSING
            td001 =td001.rename(columns={'id':'td001_dpe_id'})
            td011_p['type_installation_ch']=td011_p.tv025_type_installation.replace({'Chauffage Individuel':'individuel',
                                        'Chauffage Collectif':'collectif'})
            break

D:\data\dpe_full\depts\94
D:\data\dpe_full\depts\94


# dump new

In [405]:
import main
import generate_dpe_annexes_scripts.conversion_normalisation
import generate_dpe_annexes_scripts.td011_td012_processing
import generate_dpe_annexes_scripts.td013_td014_processing
import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict

In [420]:
reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)
reload_package(main)
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)

reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)

In [421]:
from main import *

In [422]:
td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                td011, td012,
                                                                                                td013, td014)
round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')

In [15]:
td001_sys_ch.type_installation_ch_concat=td001_sys_ch.type_installation_ch_concat.str.replace('Chauffage Individuel',
                                                                                              'individuel').str.replace('Chauffage Collectif',
                                                                                                                        'collectif')
td001 =td001.rename(columns={'id':'td001_dpe_id'})

# td002/td016 processing

In [19]:
from generate_dpe_annexes_scripts.td002_td016_processing import merge_td002_td016_trtrv,extract_type_energie_from_td002_td016

In [20]:
td002_p,td016_p=merge_td002_td016_trtrv(td002,td016)
td001_ener_from_facture = extract_type_energie_from_td002_td016(td001,td002_p,td016_p)

# load dicts

In [21]:
from generate_dpe_annexes_scripts.utils import reload_package

In [107]:
import generate_dpe_annexes_scripts.text_matching_dict 
import os
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import *

from generate_dpe_annexes_scripts.conversion_normalisation import *
reload_package(generate_dpe_annexes_scripts.conversion_normalisation)
from generate_dpe_annexes_scripts.conversion_normalisation import *


['electricite',
 'gaz',
 'fioul',
 'reseau de chaleur',
 'bois',
 'autre : gpl butane propane',
 'electricite renouvelable',
 'charbon',
 'autre']

In [23]:
gen_ch_search_dict_flat.keys() -set(td012_gen_ch_search_dict.keys())

{'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'pac air/eau en releve de chaudiere',
 'pac eau/eau en releve de chaudiere',
 'pac geothermique en releve de chaudiere',
 'radiateurs gaz'}

In [24]:
set(td014_gen_ecs_search_dict.keys())-gen_ecs_search_dict_flat.keys()

{'ecs thermodynamique electrique(pompe a chaleur ou ballon)'}

In [25]:
gen_ecs_search_dict_flat.keys() -set(td014_gen_ecs_search_dict.keys())

{'abscence ecs solaire',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'chauffe-eau independant indetermine',
 'ecs solaire',
 'ecs thermodynamique electrique(pac ou ballon)',
 'production mixte fioul',
 'production mixte gaz',
 'production mixte indetermine'}

# search ECS

In [337]:

td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.fillna('indetermine')

td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_desc_txt(x))



In [338]:


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict)

energie_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [339]:
td003_ecs = td003.loc[td003.tr007_type_descriptif_id == '10']
td003_ecs.descriptif = td003_ecs.descriptif.fillna('indetermine')
td003_ecs.descriptif = td003_ecs.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ecs.descriptif = td003_ecs.descriptif.apply(lambda x:clean_desc_txt(x))



In [340]:

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_desc=m.merge(td003_ecs[['id','td001_dpe_id']])

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])



m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=energie_dict)

energie_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# search Chauffage

In [341]:

td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')

td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [342]:

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)

gen_ch_lib_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict)

energie_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [343]:
td003_ch = td003.loc[td003.tr007_type_descriptif_id == '11']
td003_ch.descriptif = td003_ch.descriptif.fillna('indetermine')
td003_ch.descriptif = td003_ch.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ch.descriptif = td003_ch.descriptif.apply(lambda x:clean_desc_txt(x))
  

In [345]:

m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)
gen_ch_lib_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=energie_dict)

energie_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# fusion

## méthodologie de fusion

TBD

In [435]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id','tr002_type_batiment_id']].merge(td001_sys_ch[['td001_dpe_id',
                                                                                             'gen_ch_lib_infer_concat',
                                                                                            ]],on='td001_dpe_id',
                                                                              how='left')
td001_sys=td001_sys.merge(td001_sys_ecs[['td001_dpe_id','gen_ecs_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')

## Chauffage


### préparation de la donnée

#### generateur chauffage

In [436]:
# libéllés des generateurs chauffage issues des données textes
gen_ch_lib_from_txt=pd.concat([gen_ch_lib_desc[['label','category','td001_dpe_id']],
                               gen_ch_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ch_lib_from_txt =gen_ch_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ch_lib_from_txt = gen_ch_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

gen_ch_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ch_lib_from_data = td012_p[['td001_dpe_id','gen_ch_lib_infer']].copy()

gen_ch_lib_from_data.columns=['td001_dpe_id','label']

gen_ch_lib_from_data['category']=gen_ch_lib_from_data.label.replace(reverse_cat_gen_ch)

gen_ch_lib_from_data['label']=pd.Categorical(gen_ch_lib_from_data.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ch_lib_from_data['source']='data'

# concat. 
gen_ch_lib = pd.concat([gen_ch_lib_from_data,gen_ch_lib_from_txt],axis=0)

gen_ch_lib['source']=pd.Categorical(gen_ch_lib.source,categories=['data','txt'],ordered=True)
gen_ch_lib['category']=pd.Categorical(gen_ch_lib.category,categories=list(gen_ch_search_dict.keys()),ordered=True)



#### type installation

In [437]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ch_from_txt = pd.concat([type_installation_ch_desc[['label','td001_dpe_id']],
                               type_installation_ch_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ch_from_txt['label'] = pd.Categorical(type_installation_ch_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ch_from_txt = type_installation_ch_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ch_from_txt = type_installation_ch_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ch_from_txt['source'] = 'txt'

type_installation_ch_from_data = td011_p[['td001_dpe_id','type_installation_ch']].copy()
type_installation_ch_from_data.columns = ['td001_dpe_id','label']
type_installation_ch_from_data['label'] = type_installation_ch_from_data['label'].fillna('indetermine')
type_installation_ch_from_data['label'] = pd.Categorical(type_installation_ch_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ch_from_data['source'] = 'data'
type_installation_ch = pd.concat([type_installation_ch_from_data,type_installation_ch_from_txt],axis=0)


type_installation_ch['is_indetermine']=type_installation_ch.label=='indetermine'

#### energie

In [438]:


# libéllés des generateurs chauffage issues des données textes
energie_ch_from_txt = pd.concat([energie_ch_desc[['label','td001_dpe_id']],
                               energie_ch_ft[['label','td001_dpe_id']]],axis=0)

energie_ch_from_txt['label'] = pd.Categorical(energie_ch_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ch_from_txt = energie_ch_from_txt.sort_values(by=['td001_dpe_id','label'])

energie_ch_from_txt = energie_ch_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

energie_ch_from_txt['source'] = 'txt'

energie_ch_from_data = td012_p[['td001_dpe_id','type_energie_ch']].copy()
energie_ch_from_data.columns = ['td001_dpe_id','label']
energie_ch_from_data['label'] = energie_ch_from_data['label'].fillna('indetermine')
energie_ch_from_data['label'] = pd.Categorical(energie_ch_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ch_from_data['source'] = 'data'
energie_ch = pd.concat([energie_ch_from_data,energie_ch_from_txt],axis=0)


energie_ch['is_indetermine']=energie_ch.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [439]:
is_chaudiere_ener_ind = gen_ch_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ch_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ch_lib.label.str.contains('chaudiere')&gen_ch_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ch_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ch_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ch_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ind
chaudiere_ind=gen_ch_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ch_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### fusion

#### generateur chauffage

In [440]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])


gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ch_lib = gen_ch_lib.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ch_lib=gen_ch_lib.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ch_lib = gen_ch_lib.to_frame('gen_ch_lib_final')

td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')

diff=td001_sys.gen_ch_lib_infer_concat!=td001_sys.gen_ch_lib_final

ind=td001_sys.loc[td001_sys.gen_ch_lib_final=='chaudiere energie indetermine condensation']

#### type installation

In [441]:
type_installation_ch = type_installation_ch.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ch = type_installation_ch.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ch = type_installation_ch.set_index('td001_dpe_id').label.to_frame('type_installation_ch')

td001_sys = td001_sys.merge(type_installation_ch.reset_index(),on='td001_dpe_id',how='left')


#### energie

In [442]:

energie_ch = energie_ch.sort_values(by=['td001_dpe_id','label','source'])

energie_ch = energie_ch.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ch=energie_ch.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ch')




td001_sys = td001_sys.merge(energie_ch.reset_index(),on='td001_dpe_id',how='left')


#  ecs

### preparation de la donnée


#### generateur ecs

In [444]:


# libéllés des generateurs chauffage issues des données textes
gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ecs_lib_from_txt =gen_ecs_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ecs_lib_from_txt = gen_ecs_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

gen_ecs_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ecs_lib_from_data = td014_p[['td001_dpe_id','gen_ecs_lib_infer']].copy()

gen_ecs_lib_from_data.columns=['td001_dpe_id','label']
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')
gen_ecs_lib_from_data['category']=gen_ecs_lib_from_data.label.replace(reverse_cat_gen_ecs)

gen_ecs_lib_from_data['label']=pd.Categorical(gen_ecs_lib_from_data.label,categories=list(gen_ecs_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')

gen_ecs_lib_from_data['source']='data'

# concat. 
gen_ecs_lib = pd.concat([gen_ecs_lib_from_data,gen_ecs_lib_from_txt],axis=0)

gen_ecs_lib['source']=pd.Categorical(gen_ecs_lib.source,categories=['data','txt'],ordered=True)
gen_ecs_lib['category']=pd.Categorical(gen_ecs_lib.category,categories=list(gen_ecs_search_dict.keys()),ordered=True)



#### type installation

In [445]:


# libéllés des generateurs chauffage issues des données textes
type_installation_ecs_from_txt = pd.concat([type_installation_ecs_desc[['label','td001_dpe_id']],
                               type_installation_ecs_ft[['label','td001_dpe_id']]],axis=0)

type_installation_ecs_from_txt['label'] = pd.Categorical(type_installation_ecs_from_txt['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)



type_installation_ecs_from_txt = type_installation_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

type_installation_ecs_from_txt = type_installation_ecs_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

type_installation_ecs_from_txt['source'] = 'txt'

type_installation_ecs_from_data = td014_p[['td001_dpe_id','type_installation_ecs']].copy()
type_installation_ecs_from_data.columns = ['td001_dpe_id','label']
type_installation_ecs_from_data['label'] = type_installation_ecs_from_data['label'].fillna('indetermine')
type_installation_ecs_from_data['label'] = pd.Categorical(type_installation_ecs_from_data['label'],
                                                    categories=['collectif','individuel','indetermine'],ordered=True)
type_installation_ecs_from_data['source'] = 'data'
type_installation_ecs = pd.concat([type_installation_ecs_from_data,type_installation_ecs_from_txt],axis=0)


type_installation_ecs['is_indetermine']=type_installation_ecs.label=='indetermine'

#### energie 

In [446]:


# libéllés des generateurs chauffage issues des données textes
energie_ecs_from_txt = pd.concat([energie_ecs_desc[['label','td001_dpe_id']],
                               energie_ecs_ft[['label','td001_dpe_id']]],axis=0)

energie_ecs_from_txt['label'] = pd.Categorical(energie_ecs_from_txt['label'],
                                                    categories=energie_normalise_ordered,ordered=True)



energie_ecs_from_txt = energie_ecs_from_txt.sort_values(by=['td001_dpe_id','label'])

energie_ecs_from_txt = energie_ecs_from_txt.drop_duplicates(subset=['td001_dpe_id'],keep='first')

energie_ecs_from_txt['source'] = 'txt'

energie_ecs_from_data = td014_p[['td001_dpe_id','type_energie_ecs']].copy()
energie_ecs_from_data.columns = ['td001_dpe_id','label']
energie_ecs_from_data['label'] = energie_ecs_from_data['label'].fillna('indetermine')
energie_ecs_from_data['label'] = pd.Categorical(energie_ecs_from_data['label'],
                                                    categories=energie_normalise_ordered,ordered=True)
energie_ecs_from_data['source'] = 'data'
energie_ecs = pd.concat([energie_ecs_from_data,energie_ecs_from_txt],axis=0)


energie_ecs['is_indetermine']=energie_ecs.label=='indetermine'

### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [447]:
is_chaudiere_ener_ind = gen_ecs_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ecs_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ecs_lib.label.str.contains('chaudiere')&gen_ecs_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ecs_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ecs_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ecs_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ind
chaudiere_ind=gen_ecs_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ecs_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### fusion

In [448]:
priorisation_ecs={'chaudiere':'principal',
 'effet_joule':'secondaire',
"production_mixte_indetermine":"secondaire",
 'chauffe-eau_independant':'defaut',
 'reseau de chaleur':'principal',
 'solaire':'solaire',
 'abscence_solaire':'solaire',
 'ecs_thermodynamique':'principal',
'indetermine':'defaut'}

In [449]:
gen_ecs_lib['priorite']=gen_ecs_lib.category.replace(priorisation_ecs)

#### traitement des generateurs d'ECS prioritaires

In [451]:
gen_ecs_lib_princ = gen_ecs_lib.query('priorite=="principal"')

In [452]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id']].merge(td001_sys_ch[['td001_dpe_id','gen_ch_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')


td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')

In [453]:


gen_ecs_lib_princ = gen_ecs_lib_princ.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_princ = gen_ecs_lib_princ.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_princ=gen_ecs_lib_princ.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_princ = gen_ecs_lib_princ.to_frame('gen_ecs_lib_final')



td001_sys = td001_sys.merge(gen_ecs_lib_princ.reset_index(),on='td001_dpe_id',how='left')


#### traitement des generateurs d'ecs secondaires

In [454]:
gen_ecs_lib_sec = gen_ecs_lib.query('priorite=="secondaire"')

gen_ecs_lib_sec = gen_ecs_lib_sec.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_sec = gen_ecs_lib_sec.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_sec=gen_ecs_lib_sec.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_sec = gen_ecs_lib_sec.to_frame('gen_ecs_lib_final_sec')

td001_sys = td001_sys.merge(gen_ecs_lib_sec.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_sec']

#### traitement des generateurs d'ecs par défaut

In [455]:
gen_ecs_lib_defaut = gen_ecs_lib.query('priorite=="defaut"')

gen_ecs_lib_defaut = gen_ecs_lib_defaut.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ecs_lib_defaut = gen_ecs_lib_defaut.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ecs_lib_defaut=gen_ecs_lib_defaut.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ecs_lib_defaut = gen_ecs_lib_defaut.to_frame('gen_ecs_lib_final_defaut')

td001_sys = td001_sys.merge(gen_ecs_lib_defaut.reset_index(),on='td001_dpe_id',how='left')

null=td001_sys.gen_ecs_lib_final.isnull()

td001_sys.loc[null,'gen_ecs_lib_final']=td001_sys.loc[null,'gen_ecs_lib_final_defaut']

####  installation ecs

In [456]:
type_installation_ecs = type_installation_ecs.sort_values(by=['td001_dpe_id','is_indetermine',
                                     'source','label'])
type_installation_ecs = type_installation_ecs.drop_duplicates(subset=['td001_dpe_id'])

type_installation_ecs = type_installation_ecs.set_index('td001_dpe_id').label.to_frame('type_installation_ecs')

td001_sys = td001_sys.merge(type_installation_ecs.reset_index(),on='td001_dpe_id',how='left')


####  energie ecs

In [457]:

energie_ecs = energie_ecs.sort_values(by=['td001_dpe_id','label','source'])

energie_ecs = energie_ecs.drop_duplicates(subset=['td001_dpe_id','label'])

energie_ecs=energie_ecs.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x))).to_frame('type_energie_ecs')




td001_sys = td001_sys.merge(energie_ecs.reset_index(),on='td001_dpe_id',how='left')


## retraitement des energies non affectées pour les chaudières avec les energies factures

In [458]:
td001_sys =td001_sys.merge(td001_ener_from_facture,on='td001_dpe_id',how='left')


### chauffage

In [102]:


ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in ['autre : gpl butane propane','charbon','fioul','gaz']:
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())

8360
8360
8360
8360
7963
7907
3088
2682


### ECS

In [103]:


ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')

for energie in reversed(energie_chaudiere_mods):
    ener = td001_sys.td002_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())
    ener = td001_sys.td016_type_energie_ecs.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ecs_lib_final']=td001_sys.loc[ener&ind,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ecs_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())

8034
8034
8034
8034
7806
7763
4309
3933


## cohérence sys ecs chauffage

In [104]:
td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']]=td001_sys[['gen_ch_lib_final','gen_ecs_lib_final']].fillna('indetermine')

In [105]:
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

### chaudieres

In [106]:
is_chaudiere_ener_ind_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere energie indetermine')
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_chaudiere_ecs = td001_sys.gen_ecs_lib_final.str.contains('chaudiere')



#### affectation du système mixte à l'ECS lorsque celle ci est indetermine

In [107]:
sub_by_ch=is_ind_ecs&is_chaudiere_ch

In [108]:
td001_sys.loc[sub_by_ch,'gen_ecs_lib_final']=td001_sys.loc[sub_by_ch,'gen_ch_lib_final']

#### correction et convergence des energies de chaudieres entre ECS et chauffage

In [109]:
for energie in list(reversed(energie_chaudiere_mods)):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    sub_ch_ecs = is_ener_ch&(~is_ener_ecs)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace('energie indetermine',energie)
    sub_ecs_ch = is_ener_ecs&(~is_ener_ch)&is_chaudiere_ch&is_chaudiere_ecs
    td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace('energie indetermine',energie)


#### correction et convergence des types de chaudieres entre ECS et chauffage

In [110]:
for energie in reversed(energie_chaudiere_mods):
    is_ener_ch = (td001_sys.gen_ch_lib_final.str.contains(energie))
    is_ener_ecs = (td001_sys.gen_ecs_lib_final.str.contains(energie))
    both_ener = is_ener_ch&is_ener_ecs&is_chaudiere_ch&is_chaudiere_ecs
    for type_chaudiere in reversed(type_chaudiere_mods):
        is_type_ch = (td001_sys.gen_ch_lib_final.str.contains(type_chaudiere))
        is_type_ecs = (td001_sys.gen_ecs_lib_final.str.contains(type_chaudiere))
        chaudiere_ind = f'chaudiere {energie} indetermine'
        chaudiere_type=f'chaudiere {energie} {type_chaudiere}'
        is_ind_ch = (td001_sys.gen_ch_lib_final.str.contains(chaudiere_ind))
        is_ind_ecs = (td001_sys.gen_ecs_lib_final.str.contains(chaudiere_ind))
        sub_ch_ecs = is_type_ch&both_ener&is_ind_ecs
        td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final']=td001_sys.loc[sub_ch_ecs,'gen_ecs_lib_final'].str.replace(chaudiere_ind,chaudiere_type)
        sub_ecs_ch = is_type_ecs&both_ener&is_ind_ch
        td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final']=td001_sys.loc[sub_ecs_ch,'gen_ch_lib_final'].str.replace(chaudiere_ind,chaudiere_type)

        

#### correction cas ECS affecté à un chauffe eau indépendant lorsque chaudiere chauffage

In [111]:
# refresh
is_chaudiere_ch = td001_sys.gen_ch_lib_final.str.contains('chaudiere')
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')

In [112]:
td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ecs_lib_final']=td001_sys.loc[is_chaudiere_ch&is_ind,'gen_ch_lib_final']

#### correction cas ecs si gen_ch_lib elec ou poele

In [113]:
is_ind = td001_sys.gen_ecs_lib_final.str.contains('chauffe-eau independant indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final=='indetermine')
is_ind = is_ind |(td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine'))

In [114]:
is_ch_decentr = ~td001_sys.gen_ch_lib_final.str.contains('|'.join(energie_chaudiere_mods))
is_ch_decentr = is_ch_decentr& td001_sys.gen_ch_lib_final.str.contains('elec|pac|joule|jonction|poele')

In [115]:
td001_sys.loc[is_ind&is_ch_decentr,'gen_ecs_lib_final']='ecs electrique indetermine'

In [116]:
is_ind =(td001_sys.gen_ecs_lib_final=='indetermine')

### reseaux de chaleur

In [117]:
# refresh
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [118]:
is_reseau_ch = td001_sys.gen_ch_lib_final.str.contains('reseau de chaleur')
is_reseau_ecs = td001_sys.gen_ecs_lib_final.str.contains('reseau de chaleur')

td001_sys.loc[is_ind_ch&is_reseau_ecs,'gen_ch_lib_final']='reseau de chaleur'
td001_sys.loc[is_ind_ecs&is_reseau_ch,'gen_ecs_lib_final']='reseau de chaleur'

### thermodynamique

In [119]:
# refresh
is_ind_ecs = td001_sys.gen_ecs_lib_final.str.contains('production mixte indetermine')
is_ind_ecs = is_ind_ecs | (td001_sys.gen_ecs_lib_final=="indetermine")
is_ind_ch = td001_sys.gen_ch_lib_final=="indetermine"

In [120]:
is_pac_ch = td001_sys.gen_ch_lib_final.str.contains('pac')&(~td001_sys.gen_ch_lib_final.str.contains('pac air/air'))

td001_sys.loc[is_ind_ecs&is_pac_ch,'gen_ecs_lib_final']='ecs thermodynamique electrique(pac ou ballon)'

# nettoyage final

In [121]:
td001_sys['gen_ch_lib_final']=td001_sys.gen_ch_lib_final.str.replace(' \+ indetermine','')
td001_sys['gen_ecs_lib_final']=td001_sys.gen_ecs_lib_final.str.replace(' \+ indetermine','')



In [122]:
td001_sys.loc[diff&is_chaudiere_ch&is_chaudiere_ecs][['gen_ch_lib_final','gen_ecs_lib_final','td001_dpe_id']].to_dict(orient='records')

[{'gen_ch_lib_final': 'chaudiere gaz indetermine',
  'gen_ecs_lib_final': 'chaudiere gaz indetermine',
  'td001_dpe_id': '1737'},
 {'gen_ch_lib_final': 'chaudiere gaz standard',
  'gen_ecs_lib_final': 'chaudiere gaz standard',
  'td001_dpe_id': '1989'},
 {'gen_ch_lib_final': 'chaudiere gaz standard',
  'gen_ecs_lib_final': 'chaudiere gaz standard',
  'td001_dpe_id': '1991'},
 {'gen_ch_lib_final': 'chaudiere gaz standard',
  'gen_ecs_lib_final': 'chaudiere gaz standard',
  'td001_dpe_id': '2252'},
 {'gen_ch_lib_final': 'chaudiere gaz indetermine',
  'gen_ecs_lib_final': 'chaudiere gaz indetermine',
  'td001_dpe_id': '2962'},
 {'gen_ch_lib_final': 'chaudiere gaz standard',
  'gen_ecs_lib_final': 'chaudiere gaz standard',
  'td001_dpe_id': '3071'},
 {'gen_ch_lib_final': 'chaudiere fioul indetermine',
  'gen_ecs_lib_final': 'chaudiere fioul indetermine',
  'td001_dpe_id': '3228'},
 {'gen_ch_lib_final': 'chaudiere fioul standard',
  'gen_ecs_lib_final': 'chaudiere fioul standard',
  'td001_

In [123]:
td001_sys

,td001_dpe_id,gen_ch_lib_infer_concat,gen_ch_lib_final,gen_ecs_lib_infer_concat,gen_ecs_lib_final,gen_ecs_lib_final_sec,gen_ecs_lib_final_defaut,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs
0,212,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,production mixte indetermine,NaN,autre,autre,NaN,NaN
1,304,chaudiere gaz indetermine,chaudiere gaz indetermine,chaudiere gaz indetermine,chaudiere gaz indetermine,production mixte indetermine,chauffe-eau independant indetermine,gaz,gaz,NaN,NaN
2,566,convecteurs electriques nfc,convecteurs electriques nfc,ballon a accumulation electrique,ballon a accumulation electrique,ballon a accumulation electrique,chauffe-eau independant indetermine,electricite,electricite,NaN,NaN
3,898,reseau de chaleur,reseau de chaleur,reseau de chaleur,reseau de chaleur,production mixte indetermine,NaN,autre,autre,NaN,NaN
4,1500,convecteurs electriques nfc,panneaux rayonnants electriques nfc,ballon a accumulation electrique,ballon a accumulation electrique,ballon a accumulation electrique,NaN,electricite,electricite,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
145974,9429928,NaN,chaudiere gaz indetermine,NaN,chaudiere gaz indetermine,production mixte indetermine,NaN,gaz,gaz,NaN,NaN
145975,9430782,chaudiere gaz indetermine,chaudiere gaz indetermine,chaudiere gaz indetermine,chaudiere gaz indetermine,production mixte indetermine,NaN,gaz,gaz,NaN,NaN
145976,9430797,NaN,chaudiere gaz indetermine,NaN,chaudiere gaz indetermine,NaN,chauffe-eau independant indetermine,gaz,NaN,NaN,NaN
145977,9431074,NaN,chaudiere gaz indetermine,NaN,chaudiere gaz indetermine,NaN,chauffe-eau gaz independant,gaz,NaN,gaz,NaN


In [80]:
'chaudiere fioul standard + indetermine'.replace(' + indetermine','')

'chaudiere fioul standard'

In [62]:
td001_sys.loc[td001_sys.td001_dpe_id=="9404509"].to_dict()

{'td001_dpe_id': {145657: '9404509'},
 'gen_ch_lib_infer_concat': {145657: nan},
 'gen_ch_lib_final': {145657: 'chaudiere electrique + panneaux rayonnants electriques nfc'},
 'gen_ecs_lib_infer_concat': {145657: nan},
 'gen_ecs_lib_final': {145657: 'chaudiere energie indetermine indetermine'},
 'gen_ecs_lib_final_sec': {145657: 'chaudiere electrique + production mixte indetermine'},
 'gen_ecs_lib_final_defaut': {145657: 'chauffe-eau independant indetermine'},
 'td002_type_energie_ch': {145657: nan},
 'td002_type_energie_ecs': {145657: nan},
 'td016_type_energie_ch': {145657: nan},
 'td016_type_energie_ecs': {145657: nan}}

# simplification 

# debug

In [ ]:
ids=td001_sys.query('gen_ecs_lib_final=="chaudiere energie indetermine standard + reseau de chaleur"').td001_dpe_id.unique()

gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)




m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict_flat)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')

gen_ecs_lib_ft

td001_sys.loc[td001_sys.td001_dpe_id=="8085231"]

gen_ecs_lib_ft.loc[gen_ecs_lib_ft.td001_dpe_id=='8085231']

gen_ecs_lib_from_txt.loc[gen_ecs_lib_from_txt.td001_dpe_id=='8085231']

gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id=='8085231']

td005_ch.loc[td005_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ch.loc[td003_ch.td001_dpe_id.isin(ids)].to_dict(orient='records')

td003_ecs.loc[td003_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td005_ecs.loc[td005_ecs.td001_dpe_id.isin(ids)].to_dict(orient='records')

td001_sys.gen_ecs_lib_final.fillna('NA').value_counts()

In [332]:
td005_ecs.loc[td005_ecs.td001_dpe_id.isin(null_ids)].to_dict(orient='records')


[{'id': '41243',
  'td001_dpe_id': '2124',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': ' - ecs n 1',
  'est_efface': '0'},
 {'id': '60254',
  'td001_dpe_id': '3113',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'chauffe-eau gaz installe apres 2000  puissance   10kw)  avec veilleuse<br  becs : 1132  rd : 1  rg : 0 5  pn : 19  iecs : 2 24  fecs : 0',
  'est_efface': '0'},
 {'id': '105926',
  'td001_dpe_id': '5466',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'chauffe-eau gaz installe apres 2000  avec veilleuse<br  becs : 882  rd : 1  rg : 0 5  pn : 8  iecs : 2 56  fecs : 0  vs : 5',
  'est_efface': '0'},
 {'id': '106094',
  'td001_dpe_id': '5475',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'chauffe-eau gaz installe apres 2000  avec veilleuse<br  becs : 882  rd : 1  rg : 0 5  pn : 8  iecs : 2 56  fecs : 0  vs : 5',
  'est_efface': '0'},
 {'id': '111287',
  'td001_dpe_id'

In [ ]:
td

In [ ]:
null_ids

In [158]:
id_="9429928"

In [159]:
td001_sys.query(f'td001_dpe_id=="{id_}"')

,td001_dpe_id,gen_ch_lib_infer_concat,gen_ch_lib_final,gen_ecs_lib_infer_concat,gen_ecs_lib_final,gen_ecs_lib_final_sec,gen_ecs_lib_final_defaut,type_installation_ecs,td002_type_energie_ch,td002_type_energie_ecs,td016_type_energie_ch,td016_type_energie_ecs,type_energie_ecs,type_installation_ch,type_energie_ch
145974,9429928,NaN,chaudiere gaz indetermine,NaN,production mixte indetermine,production mixte indetermine,NaN,individuel,gaz,gaz,NaN,NaN,NaN,individuel,NaN


In [160]:
td005_ecs.loc[td005_ecs.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '174077887',
  'td001_dpe_id': '9429928',
  'tr011_sous_categorie_fiche_technique_id': '17',
  'valeur_renseignee': 'idem chauffage aucun equipement individuel',
  'est_efface': '0'}]

In [161]:
td003_ch.loc[td003_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '191767078',
  'td001_dpe_id': '9429928',
  'tr007_type_descriptif_id': '11',
  'descriptif': 'chaudiere ancienne collective au gaz naturel aucun equipement individuel',
  'est_efface': '0'}]

In [162]:
td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '174077886',
  'td001_dpe_id': '9429928',
  'tr011_sous_categorie_fiche_technique_id': '16',
  'valeur_renseignee': 'chaudiere ancienne collective au gaz naturel aucun equipement individuel',
  'est_efface': '0'}]

In [163]:
td003_ecs.loc[td003_ecs.td001_dpe_id==id_].to_dict(orient='records')


[{'id': '191767079',
  'td001_dpe_id': '9429928',
  'tr007_type_descriptif_id': '10',
  'descriptif': 'idem chauffage aucun equipement individuel',
  'est_efface': '0'}]

In [172]:
gen_ch_lib_ft.loc[gen_ch_lib_ft.td001_dpe_id==id_]

,id,label,category,td001_dpe_id
148116,60253,chaudiere energie indetermine indetermine,chaudiere,3113


In [433]:
m.loc[m.id=='174001369']

,id,label


In [416]:
energie_ch_ft.loc[energie_ch_ft.td001_dpe_id==id_]

,id,label,td001_dpe_id
92363,174001369,Charbon,9425890


In [ ]:
m.loc[gen_ch_lib_ft.td001_dpe_id==id_]

In [ ]:
m

In [266]:
999852

999852

In [471]:
data_to_search = td005_ch


In [474]:
id_col='id'
val_col='valeur_renseignee'
search_dict=gen_ch_search_dict_flat

In [477]:
es_client = setup_es_client()
bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
# res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

# res_table=res_serie.to_frame('label')

# res_table.index.name = 'id'
# res_table=res_table.reset_index()

(129490, [])

In [476]:
res_table.loc[res_table.id=='174001369']

,id,label


In [489]:
s_list = list()
for label,char_list in  search_dict.items():

    search_instruction =generate_instruction_from_list(char_list)
    search_body = {
        "query": {
            "query_string": {
                "query": search_instruction,
                "default_field": "text_to_analyze"
            },

        },

    }
    #es_client.count(index=index_name, body=search_body)

    a_dict=es_client.search(index=index_name, body=search_body,size=500000)

    hits = a_dict['hits']['hits']

    s=pd.Series(index=[el['_source']['id'] for el in hits])
    
    s[:]=label
    s_list.append(s)
s_all=pd.concat(s_list)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [482]:
[el['_source'] for el in hits if  el['_source']['id']=='174001369']

[{'id': '174001369',
  'text_to_analyze': 'poele au charbon   insert installe apres 2001 avec label flamme verte  systeme individuel)<br  '}]

In [485]:
s[:]=label

In [486]:
s

146290980    poele ou insert bois
44997988     poele ou insert bois
55337697     poele ou insert bois
57200175     poele ou insert bois
57204622     poele ou insert bois
                     ...         
12842927     poele ou insert bois
18027967     poele ou insert bois
18100042     poele ou insert bois
13645260     poele ou insert bois
17235737     poele ou insert bois
Length: 1171, dtype: object

174001369    poele ou insert bois
dtype: object

In [488]:
[el for el in s_all el =='174001369']

[]

In [481]:
hits

[{'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': '8vwkinYBjBMnNQstRzG6',
  '_score': 31.21572,
  '_source': {'id': '146290980',
   'text_to_analyze': ' - chaudiere bois atmospherique avant 1978 poele ou insert bois avant 2ooo ou sans label flamme verte'}},
 {'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': 'x_skinYBjBMnNQstOQUw',
  '_score': 30.094772,
  '_source': {'id': '44997988',
   'text_to_analyze': 'installation de chauffage avec insert ou poele bois ou biomasse en appoint  84\xa0m²)\xa0:\r - chauffage 1 : convecteurs nfc  energie : electrique)  sans equipement d intermittence\r - appoint 1 : insert  energie : bois)  equipement recent  apres 2001)  label flamme verte'}},
 {'_index': 'dpe_extract_text',
  '_type': '_doc',
  '_id': 'v_skinYBjBMnNQstOinR',
  '_score': 30.094772,
  '_source': {'id': '55337697',
   'text_to_analyze': 'installation de chauffage avec insert ou poele bois ou biomasse en appoint  120\xa0m²)\xa0:\r - chauffage 1 : radiateurs nfc  energ

# debug search

In [258]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="""
 - collectif : solaire avec appoint chaudiere a condensation gaz naturel"""

toto="""

'type d equipement : chaudiere bois classe 4 energie : bois type d installation : individuelle  sans solaire description : standard  anciennete 0 distribution : production dans le volume chauffe  pieces alimentees contigues  :  '
"""
toto ="""
'installation d ecs 34 9\xa0m²\r - ecs 1 : chauffe-eau standard  energie : electrique) avec accumulation horizontale 75\xa0l ; production en volume habitable  alimentant des pieces non contigues'
"""
toto ="""
poele au charbon   insert installe apres 2001 avec label flamme verte  systeme individuel)<br  
"""
#toto = td005_ch.loc[td005_ch.td001_dpe_id==id_].to_dict(orient='records')[1]['valeur_renseignee']

toto ='chaudiere collective gaz naturel installee entre 1991 et 2000 \remetteurs: radiateurs<br  '

In [191]:
index_name = 'dpe_extract_text'

In [259]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['ecs thermodynamique electrique(pac ou ballon)'])
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['abscence ecs solaire'])



In [260]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['chaudiere bois'])

search_instruction = generate_instruction_from_list(gen_ecs_search_dict_flat['production mixte indetermine'])
search_instruction = generate_instruction_from_list(gen_ch_search_dict_flat['poele ou insert bois'])

In [261]:
search_instruction = generate_instruction_from_list(energie_dict['gaz'])
search_instruction = generate_instruction_from_list(installation_dict['collectif'])

In [262]:
#search_instruction='((mixte) OR (combine) OR (chauffage AND ecs) OR (lie ) OR (combine ) OR (idem))'

In [263]:
#search_instruction = 'energie'

In [264]:
search_instruction

'((collective) OR (collectif) OR (coll) OR (coll.))'

In [265]:
toto

'chaudiere collective gaz naturel installee entre 1991 et 2000 \remetteurs: radiateurs<br  '

In [266]:
import time

In [267]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client()
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.2876821,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'B3YGzXYBN3qOYd9foKa9',
    '_score': 0.2876821,
    '_source': {'id': 'id',
     'text_to_analyze': 'chaudiere collective gaz naturel installee entre 1991 et 2000 \remetteurs: radiateurs<br  '}}]}}

In [289]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [60]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [53]:
es_client

<Elasticsearch([{}])>

In [54]:
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'MeKchHYBLPwkNvcNz20A',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': '\nsysteme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 \n'}}]}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')